In [ ]:
import os
import json
import sqlite3
import requests
import pandas as pd

# Configuration
LOCAL_FILE = "data/local_data.csv"
API_URL = "https://api.publicapis.org/entries"  # Example API

# Function to fetch remote data (API)
def fetch_remote_data(api_url):
    try:
        response = requests.get(api_url)
        response.raise_for_status()  # Raise error for bad response
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

# Function to load local CSV or JSON
def load_local_data(file_path):
    try:
        if file_path.endswith(".csv"):
            return pd.read_csv(file_path)
        elif file_path.endswith(".json"):
            return pd.read_json(file_path)
        else:
            raise ValueError("Unsupported file format")
    except Exception as e:
        print(f"Error loading local data: {e}")
        return None

# Function to transform data (convert format, modify columns)
def transform_data(df, output_format="csv"):
    try:
        df["new_column"] = "Sample Data"  # Example transformation
        if output_format == "csv":
            df.to_csv("output/transformed_data.csv", index=False)
        elif output_format == "json":
            df.to_json("output/transformed_data.json", orient="records")
        return df
    except Exception as e:
        print(f"Error during transformation: {e}")
        return None

# Function to store data in SQLite
def store_in_database(df, db_name="data_store.db", table_name="dataset"):
    try:
        conn = sqlite3.connect(db_name)
        df.to_sql(table_name, conn, if_exists="replace", index=False)
        conn.close()
        print(f"Data stored successfully in {db_name}")
    except Exception as e:
        print(f"Error storing data: {e}")

# Function to summarize data
def summarize_data(df):
    try:
        summary = {
            "num_records": len(df),
            "num_columns": len(df.columns),
            "columns": list(df.columns),
        }
        print(json.dumps(summary, indent=4))
    except Exception as e:
        print(f"Error summarizing data: {e}")

# Main ETL process
def run_etl():
    print("Starting ETL process...")

    # Extract
    print("Fetching remote data...")
    remote_data = fetch_remote_data(API_URL)
    if remote_data:
        remote_df = pd.DataFrame(remote_data["entries"])  # Example structure

    print("Loading local data...")
    local_df = load_local_data(LOCAL_FILE)

    # Transform
    if local_df is not None:
        print("Transforming local data...")
        transformed_df = transform_data(local_df)

        # Merge example
        if remote_data:
            merged_df = pd.concat([transformed_df, remote_df], axis=0, ignore_index=True)

            # Summarize
            print("Summary of merged data:")
            summarize_data(merged_df)

            # Load to database
            print("Storing data in database...")
            store_in_database(merged_df)

    print("ETL process completed.")

if __name__ == "__main__":
    run_etl()


In [ ]:
import sqlite3

# Create and initialize database
def create_database(db_name="data_store.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Example table structure
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS dataset (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            column1 TEXT,
            column2 TEXT
        )
    """)

    conn.commit()
    conn.close()
    print(f"Database {db_name} initialized.")

if __name__ == "__main__":
    create_database()
